In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import pickle
from pathlib import Path

# ==================== 1. CONFIGURACIÓN ====================
INPUT_FILE = '../data/processed/f1_features_complete.csv'
OUTPUT_DIR = '../models'

# TUS PARÁMETROS GANADORES (Copiados de tu resultado 4.5)
# Fíjate en la diferencia de profundidad: de 2 (conservador) a 6 (agresivo)
params_early = {
    'subsample': 0.8, 'reg_lambda': 5.0, 'reg_alpha': 0.1, 
    'n_estimators': 150, 'max_depth': 2, 'learning_rate': 0.01, 
    'colsample_bytree': 0.5, 'n_jobs': -1, 'random_state': 42
}

params_mid = {
    'subsample': 0.95, 'reg_lambda': 0.1, 'reg_alpha': 0.1, 
    'n_estimators': 50, 'max_depth': 2, 'learning_rate': 0.1, 
    'colsample_bytree': 0.6, 'n_jobs': -1, 'random_state': 42
}

params_late = {
    'subsample': 0.8, 'reg_lambda': 5.0, 'reg_alpha': 0, 
    'n_estimators': 150, 'max_depth': 6, 'learning_rate': 0.08, 
    'colsample_bytree': 0.8, 'n_jobs': -1, 'random_state': 42
}

# FEATURES EXACTAS (Según tu dataset actual)
feature_cols = [
    'pct_puntos_actual', 
    'pct_linear_points', 
    'posicion_media', 
    'diff_con_lider_normalizada', 
    'progreso_temporada', 
    'driver_quality_3y', 
    'team_avg_pos_3y', 
    'team_trend'
]
target_col = 'pct_puntos_final'

# ==================== 2. CARGAR DATOS ====================
print("Cargando dataset histórico completo...")
df = pd.read_csv(INPUT_FILE)
df = df.sort_values(['year', 'round', 'driver']).reset_index(drop=True)

# ==================== 3. ENTRENAMIENTO ESPECIALIZADO ====================
print("\n🏎️ ENTRENANDO SISTEMA HÍBRIDO FINAL (2008-2024)...")

# Diccionario para iterar
config_fases = {
    'early': {'data': df[df['round'] <= 5], 'params': params_early},
    'mid':   {'data': df[(df['round'] > 5) & (df['round'] <= 12)], 'params': params_mid},
    'late':  {'data': df[df['round'] > 12], 'params': params_late}
}

modelos_finales = {}

for nombre, config in config_fases.items():
    data_fase = config['data']
    params_fase = config['params']
    
    print(f"\nTraining Fase {nombre.upper()} ({len(data_fase)} muestras)...")
    print(f"  ⚙️ Perfil: Depth={params_fase['max_depth']}, L.Rate={params_fase['learning_rate']}")
    
    X = data_fase[feature_cols]
    y = data_fase[target_col]
    
    # Entrenar modelo maestro
    model = xgb.XGBRegressor(**params_fase)
    model.fit(X, y)
    
    modelos_finales[nombre] = model
    print("  ✅ Modelo entrenado.")

# ==================== 4. GUARDAR TODO ====================
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

# Guardar modelos
for nombre, model in modelos_finales.items():
    filename = f'{OUTPUT_DIR}/xgboost_{nombre}_v6_final.pkl'
    with open(filename, 'wb') as f:
        pickle.dump(model, f)
    print(f"💾 Guardado: {filename}")

# Guardar lista de features (Vital para la App)
with open(f'{OUTPUT_DIR}/final_features_list_3hiper.pkl', 'wb') as f:
    pickle.dump(feature_cols, f)

print("\n¡SISTEMA COMPLETADO! Los 3 modelos están listos para la App.")

Cargando dataset histórico completo...

🏎️ ENTRENANDO SISTEMA HÍBRIDO FINAL (2008-2024)...

Training Fase EARLY (1891 muestras)...
  ⚙️ Perfil: Depth=2, L.Rate=0.01
  ✅ Modelo entrenado.

Training Fase MID (2645 muestras)...
  ⚙️ Perfil: Depth=2, L.Rate=0.1
  ✅ Modelo entrenado.

Training Fase LATE (3043 muestras)...
  ⚙️ Perfil: Depth=6, L.Rate=0.08
  ✅ Modelo entrenado.
💾 Guardado: ../models/xgboost_early_v6_final.pkl
💾 Guardado: ../models/xgboost_mid_v6_final.pkl
💾 Guardado: ../models/xgboost_late_v6_final.pkl

¡SISTEMA COMPLETADO! Los 3 modelos están listos para la App.
